In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.utils.data as data
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split
from torch.utils.data import DataLoader, ConcatDataset
import torch
from torchvision import datasets
from torchvision.transforms import transforms

In [15]:
class VGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


In [4]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

In [5]:
dataset1 = ImageFolder("leapGestRecog",transform=train_transforms)

In [6]:
transforms2=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [7]:
dataset2 = ImageFolder("leapGestRecog",transform=transforms2)

In [10]:
data_file=data.ConcatDataset([dataset1, dataset2])

In [11]:
train=int(0.75 * len(data_file))
test=int(0.15 * len(data_file))
val=int(0.1 * len(data_file))
training_set,testing_set,validation_set=torch.utils.data.random_split(data_file, [train,test,val])

In [12]:
train_loader=torch.utils.data.DataLoader(training_set,batch_size=256)
test_loader=torch.utils.data.DataLoader(testing_set,batch_size=256)
val_loader=torch.utils.data.DataLoader(validation_set,batch_size=256)

In [13]:
def train_model(epochs,model,train_load,val_load,loss_func,gradient_descent):
    train_loss=[]
    training_accuracy=[]
    validation_loss=[]
    validation_accuracy=[]
    testing_accuracy=[]
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model=model.to(device)
    for i in range(epochs):
        training_loss=0
        correct_predicted=0
        model.train()
        for image,label in train_load:
            image,label=image.to(device),label.to(device)
            output=model(image)
            loss=loss_func(output,label)
            gradient_descent.zero_grad()
            loss.backward()
            gradient_descent.step()
            training_loss+=loss.item()*len(image)
            dummy,predicted=torch.max(output.data,1)
            correct_predicted+=(predicted==label).sum().item()
        train_loss.append(training_loss/len(train_load.dataset))
        training_accu=correct_predicted/len(train_load.dataset)
        training_accuracy.append(training_accu)


        model.eval()
        validate_loss=0
        val_correct_predicted=0
        for image,label in val_load:
            image,label=image.to(device),label.to(device)
            output=model(image)
            loss=loss_func(output,label)
            gradient_descent.zero_grad()
            loss.backward()
            gradient_descent.step()
            validate_loss+=loss.item()*len(image)
            dummy,predicted=torch.max(output.data,1)
            val_correct_predicted+=(predicted==label).sum().item()
        validation_loss.append(validate_loss/len(val_load.dataset))
        validation_accu=val_correct_predicted/len(val_load.dataset)
        validation_accuracy.append(validation_accu)
        print(f'Epoch: {i+1} --------- Train Loss: {training_loss/len(train_load.dataset):.4f} --------- Train Accuracy: {training_accu:.4f} --------- Validation Loss: {validate_loss/len(val_load.dataset):.4f} --------- Validation Accuracy: {validation_accu:.4f}')

    return model,train_loss,training_accuracy,validation_loss,validation_accuracy

In [14]:
def accuracy(test_loader, model):
    test_correct_predicted=0
    model.eval()
    with torch.no_grad():
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model=model.to(device)
        for image,label in test_loader:
            image,label=image.to(device),label.to(device)
            output=model(image)
            dummy,predicted=torch.max(output.data,1)
            test_correct_predicted+=(predicted==label).sum().item()
    testing_accu=test_correct_predicted/len(test_loader.dataset)         
    return testing_accu

In [16]:
model2=VGG16(10)
loss_function=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model2.parameters(),lr=1e-4)
returns = train_model(epochs=10,model=model2,train_load=train_loader,val_load=val_loader,loss_func=loss_function,gradient_descent=optimizer)